In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_core.prompts import PromptTemplate

from langchain_classic.chains.retrieval_qa.base import RetrievalQA


In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
import os
# 设置http的代理和不代理的地址
os.environ['HTTP_PROXY'] = "http://127.0.0.1:7890"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:7890"
os.environ['NO_PROXY'] = "http://127.0.0.1:11434" #ollama的本地服务地址

In [4]:
## Read the ppdfs from the folder
loader=PyPDFDirectoryLoader("papers")

documents=loader.load()

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)

final_documents=text_splitter.split_documents(documents)
final_documents[0]

Document(metadata={'producer': 'pdfTeX-1.40.21', 'creator': 'LaTeX with hyperref', 'creationdate': '2021-06-02T00:38:11+00:00', 'author': '', 'keywords': '', 'moddate': '2021-06-02T00:38:11+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.21 (TeX Live 2020) kpathsea version 6.3.2', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'papers/Paper4.pdf', 'total_pages': 44, 'page': 0, 'page_label': '1'}, page_content='Diffusion Models Beat GANs on Image Synthesis\nPrafulla Dhariwal∗\nOpenAI\nprafulla@openai.com\nAlex Nichol∗\nOpenAI\nalex@openai.com\nAbstract\nWe show that diffusion models can achieve image sample quality superior to the\ncurrent state-of-the-art generative models. We achieve this on unconditional im-\nage synthesis by ﬁnding a better architecture through a series of ablations. For\nconditional image synthesis, we further improve sample quality with classiﬁer guid-\nance: a simple, compute-efﬁcient method for trading off diversity for ﬁdelit

In [5]:
len(final_documents)

345

In [6]:
## Embedding Using Huggingface
huggingface_embeddings=HuggingFaceEmbeddings(
    model_name="BAAI/bge-small-en-v1.5",      #sentence-transformers/all-MiniLM-l6-v2
    model_kwargs={'device':'cpu'},
    encode_kwargs={'normalize_embeddings':True}
)

In [7]:
import  numpy as np
print(np.array(huggingface_embeddings.embed_query(final_documents[0].page_content)))
print(np.array(huggingface_embeddings.embed_query(final_documents[0].page_content)).shape)

[-5.28379381e-02 -2.95848940e-02 -1.65011603e-02 -6.97876606e-03
  4.93186377e-02  3.56690958e-02 -7.25341365e-02 -6.28811345e-02
 -3.42325168e-03  2.99070086e-02  1.10447528e-02 -3.16040367e-02
  3.81746106e-02  5.44714481e-02  1.93874687e-02  7.04315081e-02
 -8.44008289e-03 -1.88588444e-02  4.77212183e-02 -1.40712420e-02
  1.00122914e-02 -4.74376492e-02  4.91017736e-02 -3.02354526e-02
  6.28539827e-03  8.31162371e-03  7.16581494e-02 -6.14517853e-02
 -1.80430245e-02 -2.34286398e-01  4.32564057e-02  1.20113594e-02
  4.18742485e-02 -3.28857899e-02 -2.13135747e-04 -6.92436355e-04
 -2.48747766e-02 -2.50455774e-02 -2.86248066e-02  4.37693819e-02
 -2.68596802e-02  1.10380836e-02 -6.44884631e-02 -2.78332494e-02
  4.22788821e-02 -1.22138187e-02 -5.35549149e-02 -1.87554117e-02
 -5.26116565e-02 -1.19410744e-02  5.90018113e-04 -5.63122295e-02
 -1.46162538e-02  4.74096872e-02  1.33543573e-02  2.75178496e-02
  6.82152212e-02  2.61006635e-02  3.57025117e-02  3.64448689e-02
 -1.32619971e-02  3.70330

In [8]:
## VectorStore Creation
vectorstore=FAISS.from_documents(final_documents[:120],huggingface_embeddings)

In [9]:
## Query using Similarity Search
query="WHAT IS HEALTH INSURANCE COVERAGE?"
relevant_docments=vectorstore.similarity_search(query)

print(relevant_docments[0].page_content)

Model FID sFID Prec Rec
LSUN Bedrooms 256×256
DCTransformer†[42] 6.40 6.66 0.44 0.56
DDPM [25] 4.89 9.07 0.60 0.45
IDDPM [43] 4.24 8.21 0.62 0.46
StyleGAN [27] 2.35 6.62 0.59 0.48
ADM (dropout) 1.90 5.59 0.66 0.51
LSUN Horses 256×256
StyleGAN2 [28] 3.84 6.46 0.63 0.48
ADM 2.95 5.94 0.69 0.55
ADM (dropout) 2.57 6.81 0.71 0.55
LSUN Cats 256×256
DDPM [25] 17.1 12.4 0.53 0.48
StyleGAN2 [28] 7.25 6.33 0.58 0.43
ADM (dropout) 5.57 6.69 0.63 0.52
ImageNet 64×64
BigGAN-deep* [5] 4.06 3.96 0.79 0.48
IDDPM [43] 2.92 3.79 0.74 0.62
ADM 2.61 3.77 0.73 0.63
ADM (dropout) 2.07 4.29 0.74 0.63
Model FID sFID Prec Rec
ImageNet 128×128
BigGAN-deep [5] 6.02 7.18 0.86 0.35
LOGAN†[68] 3.36
ADM 5.91 5.09 0.70 0.65
ADM-G (25 steps) 5.98 7.04 0.78 0.51
ADM-G 2.97 5.09 0.78 0.59
ImageNet 256×256
DCTransformer†[42] 36.51 8.24 0.36 0.67
VQ-V AE-2†‡[51] 31.11 17.38 0.36 0.57
IDDPM‡[43] 12.26 5.42 0.70 0.62
SR3†‡[53] 11.30
BigGAN-deep [5] 6.95 7.36 0.87 0.28
ADM 10.94 6.02 0.69 0.63


In [10]:
retriever=vectorstore.as_retriever(search_type="similarity",search_kwargs={"k":3})
print(retriever)

tags=['FAISS', 'HuggingFaceEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x715357844090> search_kwargs={'k': 3}


In [11]:
from huggingface_hub import ModelCard,model_info

card = ModelCard.load("moonshotai/Kimi-K2-Thinking")
print(card.data)
print(model_info("moonshotai/Kimi-K2-Thinking").pipeline_tag)

library_name: transformers
license: other
license_name: modified-mit
text-generation


The Hugging Face Hub is an platform with over 350k models, 75k datasets, and 150k demo apps (Spaces), all open source and publicly available, in an online platform where people can easily collaborate and build ML together.

In [12]:
from langchain_huggingface import HuggingFaceEndpoint

## 这种方法已经不可用，解决方案参考 test.py
hf=HuggingFaceEndpoint(
    repo_id="meta-llama/Llama-3.1-8B-Instruct"
)
query="What is the health insurance coverage?"
# hf.invoke(query)
hf.invoke(query)
# hf.get_num_tokens(query)

ValueError: Model meta-llama/Llama-3.1-8B-Instruct is not supported for task text-generation and provider novita. Supported task: conversational.

In [13]:
#Hugging Face models can be run locally through the HuggingFacePipeline class.
from langchain_huggingface import HuggingFacePipeline

#这种方法是下载模型到本地进行运行
hf = HuggingFacePipeline.from_model_id(
    model_id="Qwen/Qwen3-0.6B",
    task="text-generation",
    pipeline_kwargs={"temperature": 0, "max_new_tokens": 300},
    model_kwargs={"trust_remote_code": True},
)

llm = hf 
llm.invoke(query)

Device set to use cpu
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


'What is the health insurance coverage? What is the difference between health insurance and health care? What is the difference between health insurance and health care? What is the difference between health insurance and health care? What is the difference between health insurance and health care? What is the difference between health insurance and health care? What is the difference between health insurance and health care? What is the difference between health insurance and health care? What is the difference between health insurance and health care? What is the difference between health insurance and health care? What is the difference between health insurance and health care? What is the difference between health insurance and health care? What is the difference between health insurance and health care? What is the difference between health insurance and health care? What is the difference between health insurance and health care? What is the difference between health insurance an

In [14]:
prompt_template="""
Use the following piece of context to answer the question asked.
Please try to provide the answer only based on the context

{context}
Question:{question}

Helpful Answers:
 """

In [15]:
# prompt=PromptTemplate(template=prompt_template,input_variables=["context","question"])
prompt=PromptTemplate.from_template(prompt_template)

In [16]:
retrievalQA=RetrievalQA.from_chain_type(
    llm=hf,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt":prompt}
)

In [17]:
query="""DIFFERENCES IN THE
UNINSURED RATE BY STATE
IN 2022"""

In [18]:
# Call the QA chain with our query.
result = retrievalQA.invoke({"query": query})
print(result['result'])


Use the following piece of context to answer the question asked.
Please try to provide the answer only based on the context

ˆq(xt+1|xt):
ˆq(xt+1|xt) =
∫
y
ˆq(xt+1,y|xt) dy (32)
=
∫
y
ˆq(xt+1|xt,y)ˆq(y|xt) dy (33)
=
∫
y
q(xt+1|xt)ˆq(y|xt) dy (34)
= q(xt+1|xt)
∫
y
ˆq(y|xt) dy (35)
= q(xt+1|xt) (36)
= ˆq(xt+1|xt,y) (37)
Following similar logic, we ﬁnd the joint distribution ˆq(x1:T|x0):
ˆq(x1:T|x0) =
∫
y
ˆq(x1:T,y|x0) dy (38)
=
∫
y
ˆq(y|x0)ˆq(x1:T|x0,y) dy (39)
=
∫
y
ˆq(y|x0)
T∏
t=1
ˆq(xt|xt−1,y) dy (40)
=
∫
y
ˆq(y|x0)
T∏
t=1
q(xt|xt−1) dy (41)
=
T∏
t=1
q(xt|xt−1)
∫
y
ˆq(y|x0) dy (42)
=
T∏
t=1
q(xt|xt−1) (43)
= q(x1:T|x0) (44)
25

producing higher ﬁdelity (but less diverse) samples.
In the above derivations, we assumed that the underlying diffusion model was unconditional, modeling
p(x). It is also possible to train conditional diffusion models, p(x|y), and use classiﬁer guidance in
the exact same way. Table 4 shows that the sample quality of both unconditional and conditional
models ca